In [6]:
import pandas as pd
import requests
import os
import re
import logging

logging.basicConfig(filename='error.log', level=logging.ERROR)

FORMAT = 'csv'
API_KEY = os.environ.get('epa_key')
DATA_FILE = 'data_list.csv'
OUTPUT_DIR = f'data'

def process_row(row):
    data_id = row['資料集代碼']
    data_name = row['資料集名稱']
    # data_freq = row['更新頻率']
    data_name = re.sub(r'[\\/:*?"<>|()（）、]', '', data_name)
    
    try:
        response = requests.get(f'https://data.epa.gov.tw/api/v2/{data_id}?format={FORMAT}&api_key={API_KEY}')
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        logging.error(f'Error retrieving data for ID {data_id}: {str(e)}')
        return
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    file_path = os.path.join(OUTPUT_DIR, f'{data_id}_{data_name}.{FORMAT}')
    with open(file_path, 'w') as f:
        f.write(response.text)

def main():
    frequency = '1日'
    df = pd.read_csv(DATA_FILE)
    df = df[df['更新頻率'] == frequency]
    df.apply(process_row, axis=1)

if __name__ == '__main__':
    main()
